In [21]:
import boto3
import json
import configparser

# Read configuration specifications

In [22]:
config = configparser.ConfigParser()
config.read('dwh.config')

KEY = config['AWS']['KEY']
SECRET = config['AWS']['SECRET']

DWH_CLUSTER_TYPE = config['DWH']['DWH_CLUSTER_TYPE']
DWH_NUM_NODES = config['DWH']['DWH_NUM_NODES']
DWH_NODE_TYPE = config['DWH']['DWH_NODE_TYPE']
DWH_CLUSTER_IDENTIFIER = config['DWH']['DWH_CLUSTER_IDENTIFIER']
DWH_DB_NAME = config['DWH']['DWH_DB_NAME']
DWH_DB_USER = config['DWH']['DWH_DB_USER']
DWH_DB_PASSWORD = config['DWH']['DWH_DB_PASSWORD']
DWH_PORT = config['DWH']['DWH_PORT']
DWH_SEC_GROUP_NAME = config['EC2']['DWH_SEC_GROUP_NAME']
DWH_IAM_ROLE_NAME = config['IAM_ROLE']['DWH_IAM_ROLE_NAME']

# Connect to required AWS resources

In [23]:
ec2_client = boto3.client('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

ec2_resource = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

# Create all resources

In [24]:
# Create IAM role
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

# Update config file
role_arn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
config['IAM_ROLE']['ARN'] = role_arn
with open('dwh.config', 'w') as conf:
    config.write(conf)

# Create security group
default_vpc = ec2_client.describe_vpcs(Filters=[{'Name':'isDefault', 'Values':['true']}])
default_vpc_id = default_vpc['Vpcs'][0]['VpcId']

try:
    redshift_sg_create = ec2_resource.create_security_group(
        Description='Authorise redshift cluster access.',
        GroupName=DWH_SEC_GROUP_NAME,
        VpcId=default_vpc_id   
    )
    redshift_sg_id = redshift_sg_create.id
except Exception as e:
    print(e)

try:
    redshift_sg = ec2_resource.SecurityGroup(redshift_sg_id)
    redshift_sg.authorize_ingress(
        GroupName=redshift_sg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

# Create cluster
try:
    response = redshift.create_cluster(
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        VpcSecurityGroupIds=[redshift_sg_id],
        
        #Roles (for s3 access)
        IamRoles=[role_arn]  
    )
except Exception as e:
    print(e)

<b><font color='red'>WAIT A FEW MINUTES FOR THE CLUSTER TO FINISH CREATING<br/> 

In [25]:
# Update config file
cluster_properties = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
host = cluster_properties['Endpoint']['Address']
config['DWH']['HOST'] = host
with open('dwh.config', 'w') as conf:
    config.write(conf)

# Delete all resources

In [26]:
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)['Cluster']['ClusterStatus']

'deleting'

<b><font color='red'>WAIT A FEW MINUTES FOR THE CLUSTER TO FINISH DELETING<br/> 

In [20]:
ec2_client.delete_security_group(GroupId=redshift_sg_id)['ResponseMetadata']['HTTPStatusCode']

200

In [19]:
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)['ResponseMetadata']['HTTPStatusCode']

200